In [2]:
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import gzip

# Changelog

## 27-07-2021

* Fix prot.accession2taxid to reflect changes of the `taxonomy.ipynb`

A new function to read the `gtdb.nr.gz` fasta headers and create only the `prot.accession2taxid.FULL.gz` file (`create_prot2taxid`)
Since the fasta itself does not contain any taxonomy information I don't need to rewrite it.
Having the genome accession available from the taxonomy files should suffice to do this independently.

* Write 
---

# Download all available proteins for all genomes

The following commands will grab everything there is to grab from the NCBI ftp, using 
the [ncbi-genome-download](https://github.com/kblin/ncbi-genome-download) utility

The `<domain>_metadata_r202.tsv` is used here, because that is what I started with. 

We can probably get the same info from the taxonomy tables used in the `taxonomy.ipynb`.

1. Create a list of accessions to download data for

* Archaea RefSeq

```
cut -f1 data/gtdb_info/ar122_metadata_r202.tsv | grep ^RS | sed -e 's/^RS_//g' > archaea_refseq.txt
```
... and download everything

```
ncbi-genome-download -s refseq -F protein-fasta -A archaea_refseq.txt -v -p 8 -o results/proteins archaea 2> archaea_refseq.log
```
---

* Achaea genbank

```
cut -f1 data/gtdb_info/ar122_metadata_r202.tsv | grep ^GB | sed -e 's/^GB_//g' > archaea_genbank.txt
```

.. and download everything

```
ncbi-genome-download -s genbank -F protein-fasta -A archaea_genbank.txt -p 8 -vv -o results/proteins archaea 2>archaea_genbank.log
```

For `Bacteria` I am running into an issue with the same approach.
It is the same as others are describing [here](https://github.com/kblin/ncbi-genome-download/issues/116).

After doing some manual checking the entries are not properly parsed, but providing them individually in the command line works.

So I ended up using `parallel` to enable multiple concurrent downloads

* Bacteria RefSeq

1. Construct the list

```
cut -f1 data/gtdb_info/bac120_metadata_r202.tsv | grep ^RS | sed -e 's/^RS_//g' > results/bacteria_refseq.txt
```

... and download

```
parallel -j8 ncbi-genome-download -s refseq -F protein-fasta -A {} -vv bacteria <results.bacteria_refseq.txt 2>results/bacteria.refseq.parallel.log
```


2. Bacteria Genbank

```
cut -f1 data/gtdb_info/bac120_metadata_r202.tsv | grep ^GB | sed -e 's/^GB_//g' > results/bacteria_genbank.txt
```

... and download

```
parallel -j8 ncbi-genome-download -s refseq -F protein-fasta -A {} -vv bacteria <results.bacteria_refseq.txt 2>results/bacteria.bacteria.parallel.log
```

----

Upon further inspection, this might be a non issue.

This 

```
ncbi-genome-download -p 8 -A results/bacteria_genbank_first10.txt --debug -F protein-fasta -s genbank -o ./test_10 bacteria
```

seems to work ok, despite all the lines printing that look like this...

```
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018711685.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018716505.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018713065.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018714105.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018712565.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018716045.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018712555.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018713505.1'
DEBUG: Skipping entry with incompatible assembly accession 'GCA_018711085.1'
```

In [5]:
# Data directories

bac_prot_refseq_dir = Path("results/proteins/refseq/bacteria")
bac_prot_genbank_dir = Path("results/proteins/genbank/bacteria")
ar_prot_refseq_dir = Path("results/proteins/refseq/archaea")
ar_prot_genbank_dir = Path("results/proteins/genbank/archaea/")

In [3]:
# Translated taxonomies from taxonomy.ipynb

names_dmp = Path("results/taxonomy/names.dmp")

In [9]:
# Raw taxonomies
# They contain genome to taxonomy mapping

archaea_tax = Path("data/gtdb_info/ar122_taxonomy_r202.tsv")
bacteria_tax = Path("data/gtdb_info/bac120_taxonomy_r202.tsv")

## Load all GTDB genome ids to taxonomy mappings in one dictionary

In [10]:
ar_tax_dic = {}
with open(archaea_tax, 'r') as fin:
    for line in fin:
        fields = [f.strip() for f in line.split('\t')]
        ar_tax_dic[fields[0]] = fields[1]

In [11]:
bac_tax_dic = {}
with open(bacteria_tax, 'r') as fin:
    for line in fin:
        fields = [f.strip() for f in line.split('\t')]
        bac_tax_dic[fields[0]] = fields[1]

In [12]:
all_tax = {**ar_tax_dic, **bac_tax_dic}

## Create a mapping of available data directories for bacteria and archaea, genbank and refseq

These are not exactly the same with GTDB. They are missing the `GB_` and `RS_` prefixes. The assembly accessions should be unique (enough..)

In [11]:
ar_prot_refseq = [d for d in ar_prot_refseq_dir.iterdir() if d.is_dir()]
ar_prot_genbank = [d for d in ar_prot_genbank_dir.iterdir() if d.is_dir()]
bac_prot_refseq = [d for d in bac_prot_refseq_dir.iterdir() if d.is_dir()]
bac_prot_genbank = [d for d in bac_prot_genbank_dir.iterdir() if d.is_dir()]

In [12]:
all_data_dirs = ar_prot_refseq + ar_prot_genbank + bac_prot_refseq + bac_prot_genbank

In [13]:
data_dirs = {d.name: d for d in all_data_dirs}

## Load in the taxonomic names produced here

In [4]:
names_dic = {}
with open(names_dmp, 'r') as fin:
    for line in fin:
        fields = [f.strip() for f in line.split('\t')]
        names_dic[int(fields[0])] = fields[2]

In [6]:
inv_names_dic = {v:k for k,v in names_dic.items()}

In [7]:
assert len(names_dic) == len(inv_names_dic)

In [17]:
def get_protein_fasta_from_dir(input_dir):
    '''
    Helper function to get proteins, if available, for a genome
    '''
    try:
        faa = list(input_dir.glob('*.faa.gz'))[0]
        return faa
    except IndexError:
#         print("No proteins found for {} (path: {})".format(input_dir.name, input_dir))
        pass

In [100]:
def transform_fasta_headers(input_fasta_gz, uid):
    seq_counter = 1
    try:
        with gzip.open(input_fasta_gz, 'rt') as fin:
            for record in SeqIO.parse(fin, "fasta"):
                new_id = "{}_{}".format(uid, seq_counter, record.description)
                transformed_record = SeqRecord(record.seq,
                                                id=new_id,
                                               description=record.description
                                              )
                seq_counter += 1

                yield transformed_record
    except EOFError:
        # It appears some downloaded files are malformatted
        print(input_fasta_gz)
        raise

In [17]:
db_dir = Path("results/db")
if not db_dir.exists():
    db_dir.mkdir(exist_ok=True)
gtdb_nr_gz = db_dir / Path("gtdb.nr.gz")
gtdb_nr_fa = db_dir / Path("gtdb.nr.fa")
prot2acc_gz = db_dir / Path("prot.accession2taxid.gz")
prot2acc = db_dir / Path("prot.accession2taxid.txt")


## Concatenate all sequences in one big file and produce the prot.accesion2taxid file

Some `faa.gz` files might be malformatted. 
This causes the generator function for the records to break.
I included a `try` - `except` clause for this specific case (`EOFError`). 
Proteins for assembly accessions that throw that, are manually redownloaded before execution of the main loop.
If this starts happening more, make a list of accessions that need to be downloaded again (loop in output dir and compare md5sum of file with MD5SUMS stored).

Edit:
After checking the log file for the downloads job, there are a few entries that have this mismatch.
They appear like:
```
ERROR: Checksum mismatch for './results/proteins/refseq/bacteria/GCF_002886255.1/GCF_002886255.1_ASM288625v1_protein.faa.gz'. Expected '8a0837198683cb1596f579101833e517', got '184c744bf6babf064d4fa623ca908cde'
```

There are also entries that look like 

```
ERROR: Download from NCBI failed: SSLError(MaxRetryError("HTTPSConnectionPool(host='ftp.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /genomes/all/GCF/002/899/015/GCF_002899015.1_ASM289901v1/md5checksums.txt (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1123)')))"))

```
Something went wrong with the connection probably.

So let's grab all these and re-download the protein files.

The following one liner will:
- Grab lines from the log file that start with 'ERROR'
- Filter out lines that contain "No downloads matched" - this is for accessions that do not appear anyway in the summary files from NCBI (why is that I don't know)
- Exactly match the "GCF_" accession string to the end of the line 
- Get only the GCF accession based on path splitting
- For some lines the complete assembly accession (e.g. `GCF_002899015.1_ASM289901v1` appear. Strip off the ASM part
- Sort and uniqify the list and output it to a file


In [103]:
!grep "^ERROR" results/bacteria.refseq.parallel.log | grep -v "No downloads matched" | grep -o "GCF_.*$" | cut -f1 -d '/' | cut -f1,2 -d'_' | sort | uniq > results/bacteria.refseq.fix.txt

Repeat the same logic for Genbank records
- Note that GC**F** changes to CC**A**


In [104]:
!grep "^ERROR.*failed" results/bacteria.genbank.parallel.log | grep -o "GCA_.*$" | cut -f1 -d'/' | cut -f1,2 -d'_' | sort | uniq > results/bacteria.genbank.fix.txt

In [105]:
!while read -r accession;do \
ncbi-genome-download -vv -s refseq -A $accession -F protein-fasta -o results/proteins bacteria; \
done <results/bacteria.refseq.fix.txt

INFO: Using cached summary.
INFO: Checking record 'GCF_000256185.2'
INFO: Using cached summary.
INFO: Checking record 'GCF_000294185.2'
INFO: Using cached summary.
INFO: Checking record 'GCF_000815645.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_000816495.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_002442495.2'
INFO: Using cached summary.
INFO: Checking record 'GCF_002445415.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_002446375.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_002886255.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_002896595.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_002899015.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_003421705.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_003422305.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_003988355.1'
INFO: Using cached summary.
INFO: Checking record 'GCF_900501045.1'


In [106]:
!while read -r accession;do \
ncbi-genome-download -vv -s genbank -A $accession -F protein-fasta -o results/proteins bacteria; \
done <results/bacteria.genbank.fix.txt

INFO: Checking record 'GCA_001817295.1'


In [107]:
missing_proteins = []
valid_proteins = []
processed_genome_ids = 0

with open(gtdb_nr_fa, 'w') as faa, open(prot2acc, 'wt') as p2acc:
    
    p2acc.write('{}\t{}\n'.format('accession.version', 'taxid'))
    
    for genome, gtdb_lineage in all_tax.items():
        genome_acc = genome[3:]
        gtdb_taxid = gtdb_lineage.split(';')[-1]
        uid = inv_names_dic[gtdb_taxid]
        
        try:
            genome_proteins = get_protein_fasta_from_dir(data_dirs[genome_acc])
            if not genome_proteins:
                missing_proteins.append(genome_acc)
            else:
                for rec in transform_fasta_headers(genome_proteins, genome):
                    SeqIO.write(rec, faa, "fasta")
                    p2acc.write("{}\t{}\n".format(rec.id, uid))
        except KeyError:
    #         print("No data dir found for {}".format(genome_acc))
            missing_proteins.append(genome_acc)
        finally:
            processed_genome_ids += 1
            if processed_genome_ids % 10 == 0:
                print("Processed {} / {} genome ids\r".format(processed_genome_ids, len(all_tax)),
                      end='')

In [109]:
processed_genome_ids

258406

In [110]:
len(missing_proteins)

43299

In [19]:
def create_prot2taxid(fasta_fp, inv_names_dic, all_taxonomies, prot2taxid_gz):
    with gzip.open(fasta_fp, 'rt') as fin, gzip.open(prot2taxid_gz, 'wt') as fout:
        fout.write('accession.version\ttaxid\n')
        for line in fin:
            if line.startswith('>'):
                # Protein id
                prot_id = line.split()[0].replace('>', '')
                # Genome accession
                genome_acc = '_'.join(prot_id.split('_')[:-1])
                
                gtdb_lineage = all_taxonomies[genome_acc]
                gtdb_name = gtdb_lineage.split(';')[-1]
                uid = inv_names_dic[gtdb_name]
                fout.write("{}\t{}\n".format(prot_id, uid))
                
                
            

In [20]:
create_prot2taxid(gtdb_nr_gz, inv_names_dic, all_tax, prot2acc_gz)